<a href="https://colab.research.google.com/github/VictorUceda/TrackNet-Padel/blob/main/TrackNet_Padel_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Mon May 23 07:53:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   67C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


import os 
path_drive = '/content/gdrive/MyDrive/Colab Notebooks/TrackNet-Padel/'
weights_path = '/content/gdrive/MyDrive/PadelModels/TrackNetWeights/TrackNet-Padel'

!pwd

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/TrackNet-Padel


In [ ]:
os.chdir('/content/')
!pwd

/content


In [ ]:
!git clone https://github.com/VictorUceda/TrackNet-Padel.git
#!git clone https://github.com/Chang-Chia-Chi/TrackNet-Badminton-Tracking-tensorflow2
os.chdir('/content/TrackNet-Padel/')
!git pull

Cloning into 'TrackNet-Padel'...
remote: Enumerating objects: 338, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 338 (delta 53), reused 0 (delta 0), pack-reused 251
Receiving objects: 100% (338/338), 139.55 MiB | 36.47 MiB/s, done.
Resolving deltas: 100% (168/168), done.
Already up to date.


In [ ]:
!pip3 install filterpy sktime piexif imutils Pillow focal_loss

### Test accuracy

In [13]:
file_name = 'Final_PremierQatar_Tie'
point_n = 0
for point_n in range(9,13):
  video = 'raw_data/points/'+file_name+"_"+str(point_n)+".mp4"
  print(video)
  csv = 'raw_data/points/'+file_name+"_"+str(point_n)+".csv"
  res = 'raw_data/points/'+file_name+"_"+str(point_n)+".resPadel.txt"
  !python ../TrackNet-Padel/predict.py --video_path={video} --label_path={csv} --load_weights={weights_path+'30'}  2>&1 | tee {res}

raw_data/points/Final_PremierQatar_Tie_9.mp4
2022-05-23 15:25:24.861009: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting......
Number of true positive: 136
Number of true negative: 65
Number of false positive FP1: 39
Number of false positive FP2: 2
Number of false negative: 146
Accuracy: 0.5180412371134021
Precision: 0.768361581920904
Recall: 0.48226950354609927
Total Time: 41.39317750930786
(ACC + Pre + Rec)/3: 0.5895574408601351
Done......
raw_data/points/Final_PremierQatar_Tie_10.mp4
2022-05-23 15:26:18.471234: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Load weights successfully
Load csv file successfully
Beginning predicting.

### Train Padel

In [ ]:
!mv raw_data/*.* raw_data/points/
for point_n in range(9):
  files = 'raw_data/points/*_'+str(point_n)+'\.*'
  print(files)
  !mv {files} raw_data/

raw_data/points/*_0\.*
raw_data/points/*_1\.*
raw_data/points/*_2\.*
raw_data/points/*_3\.*
raw_data/points/*_4\.*
raw_data/points/*_5\.*
raw_data/points/*_6\.*
raw_data/points/*_7\.*
raw_data/points/*_8\.*


In [ ]:
!rm -rf train_data/*
!python video2img.py 

==========Convert Start==========
Convert Video: raw_data/Final_PremierQatar_Tie_0.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_1.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_2.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_3.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_4.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_5.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_6.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_7.mp4
==========Convert End==========
Convert Video: raw_data/Final_PremierQatar_Tie_8.mp4
==========Convert End==========


In [12]:
!python train.py --save_weights={weights_path} --pre_trained=True  --load_weights={weights_path}

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adadelta.py:74: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adadelta, self).__init__(name, **kwargs)
2022-05-23 12:31:44.719977: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:39] Overriding allow_growth setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
Beginning training......
==========Epoch 0, Train steps: 1451, Learning rate: 0.1000==========
1451/1451 [==============================] - 457s 304ms/step - loss: 1.1532e-04 - binary_accuracy: 0.9996
==========Epoch 0 start validation==========
Epoch 0 accuracy: 0.916
Epoch 0 precision: 0.968
Epoch 0 recall: 0.941
Epoch 0 average = (accuracy + precision + recall)/3: 0.942
==========Epoch 1, Train steps: 1451, Learning rate: 0.1000==========
1451/1451 [==============================] - 441s 304ms/step - loss: 1.1514e-04 - binary_accuracy: 0.9996
==========Epoch 1 start validati